In [1]:
import pandas as pd
import requests
from io import StringIO

In [2]:
file_link = 'https://drive.google.com/file/d/1J-HpNBLEqrnGckM5bI2Dnm5hV6Vtr_FU/view?usp=drive_link'
file_link = 'https://drive.google.com/uc?id=' + file_link.split('/')[-2]

direct_download_link = file_link.replace('open', 'uc')

response = requests.get(direct_download_link)

csv_from_drive = StringIO(response.text)

In [3]:
df = pd.read_csv(csv_from_drive,
                 sep="\t",
                 names=["text", "sentiment"])

In [4]:
df.head(3)

,text,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive


In [5]:
import re

def cleansing(sent):
    string = sent.lower()
    string = re.sub(r'[^a-zA-Z0-9]',' ',string)
    return string

In [6]:
df['text_clean'] = df.text.apply(cleansing)

In [7]:
neg = df.loc[df['sentiment'] == 'negative'].text_clean.tolist()
neu = df.loc[df['sentiment'] == 'neutral'].text_clean.tolist()
pos = df.loc[df['sentiment'] == 'positive'].text_clean.tolist()

neg_sentiment = df.loc[df['sentiment'] == 'negative'].sentiment.tolist()
neu_sentiment = df.loc[df['sentiment'] == 'neutral'].sentiment.tolist()
pos_sentiment = df.loc[df['sentiment'] == 'positive'].sentiment.tolist()

In [8]:
total_data = pos + neu + neg
labels = pos_sentiment + neu_sentiment + neg_sentiment

In [9]:
total_data[0]

'warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung   tahu berkualitas   dipadu keahlian memasak   dipadu kretivitas   jadilah warung yang menyajikan menu utama berbahan tahu   ditambah menu umum lain seperti ayam   semuanya selera indonesia   harga cukup terjangkau   jangan lewatkan tahu bletoka nya   tidak kalah dengan yang asli dari tegal  '

In [10]:
#Tokenizing and Applying pad_sequences

import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(total_data)

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('tokenizer.pickle has been created.')

X = tokenizer.texts_to_sequences(total_data)
X = pad_sequences(X)

with open('x_pad_sequences.pickle','wb') as handle:
  pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('x_pad_sequences.pickle has been created.')

tokenizer.pickle has been created.
x_pad_sequences.pickle has been created.


In [11]:
len(labels)

11000

In [12]:
Y = pd.get_dummies(labels)
Y = Y.values

In [13]:
with open('y_labels.pickle','wb') as handle:
  pickle.dump(Y,handle, protocol=pickle.HIGHEST_PROTOCOL)
  print("y_labels.pickle has been created")

y_labels.pickle has been created


In [14]:
Y.shape

(11000, 3)

In [15]:
Y[0:10]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [16]:
some_df = pd.DataFrame(data=Y, columns=['a','b','c'])
some_df.head()

,a,b,c
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [17]:
some_df['labels'] = labels
some_df.head()

,a,b,c,labels
0,0,0,1,positive
1,0,0,1,positive
2,0,0,1,positive
3,0,0,1,positive
4,0,0,1,positive


In [18]:
some_df[some_df['labels']=='negative'].iloc[0:1]

,a,b,c,labels
7564,1,0,0,negative


In [19]:
some_df[some_df['labels']=='neutral'].iloc[0:1]

,a,b,c,labels
6416,0,1,0,neutral


In [20]:
some_df[some_df['labels']=='positive'].iloc[0:1]

,a,b,c,labels
0,0,0,1,positive


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
file = open("x_pad_sequences.pickle",'rb')
x = pickle.load(file)
file.close()

file = open("y_labels.pickle",'rb')
y = pickle.load(file)
file.close()

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [23]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
embed_dim = 100
units = 64

model=Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(LSTM(units, dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

adam=optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(x_train, y_train, epochs=2, batch_size=12, validation_data=(x_test, y_test), verbose=1, callbacks=[es])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 96, 100)           10000000  
                                                                 
 lstm_1 (LSTM)               (None, 64)                42240     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 10042435 (38.31 MB)
Trainable params: 10042435 (38.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None
Epoch 1/2
734/734 [==============================] - 220s 296ms/step - loss: 0.4609 - accuracy: 0.8120 - val_loss: 0.3417 - val_accuracy: 0.8732
Epoch 2/2
734/734 [==============================] - 217s 295ms/step - loss: 0.2045 - accuracy: 0.9236 - val_loss: 0.3537 - val_accuracy: 0.8686
Epoch 2: early stopping


In [26]:
from sklearn import metrics

predictions = model.predict(x_test)
y_pred = predictions
matrix_test = metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Testing selesai")
print(matrix_test)

69/69 [==============================] - 2s 16ms/step
Testing selesai
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       685
           1       0.77      0.84      0.80       233
           2       0.93      0.88      0.90      1282

    accuracy                           0.87      2200
   macro avg       0.83      0.86      0.85      2200
weighted avg       0.87      0.87      0.87      2200



In [27]:
model.save('model.h5')
print('Model has been created.')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model has been created.


In [28]:
import re
from keras.models import load_model

input_text = """
Rasa syukur, cukup.
"""

def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return string

sentiment = ['negative', 'neutral', 'positive']

text = [cleansing(input_text)]
predicted = tokenizer.texts_to_sequences(text)
guess = pad_sequences(predicted, maxlen=X.shape[1])

model = load_model('model.h5')
prediction = model.predict(guess)
polarity = np.argmax(prediction[0])

print("Text: ",text[0])
print("Sentiment: ",sentiment[polarity])

1/1 [==============================] - 0s 414ms/step
Text:   rasa syukur  cukup  
Sentiment:  negative
